# Explore CSV files and format fields to streamline the integration in a SQL DB

Most date fields in our csv are not formatted in the way MySQL expect to find them (_expected format is `YYYY-MM-DD`_). Some pre-processing is required.

**Note**: all the table from the original archive contain a comma at the end of the header. This mess up with `pandas` and other data wrangling tool. I copied the original data in files prefix with `DG_` that I use below to process the dates. _e.g._ `WATER-SYSTEM.csv` becomes `DG_WATER_SYSTEM.csv` with a first header row which DOES NOT terminate by a comma.

In [57]:
from os.path import join

import pandas as pd
from datetime import datetime as dt

**Important**: Update the global variable below with the absolute path to the folder containing all the csv files.

In [97]:
PATH_TO_DATA_FOLDER = "/Users/fpaupier/projects/safe-water/data/SDWIS/"

## `WATER_SYSTEM` table

There are 5 dates fields in this table. With two types of formatting:

1. Date formatted in `dd-mm-yy` _e.g `01-JUN-83` for June 1, 1983_. Fields encoded with this date format are:
    - `OUTSTANDING_PERFORM_BEGIN_DATE`
    - `PWS_DEACTIVATION_DATE`
    - `SOURCE_PROTECTION_BEGIN_DATE`
    
    Note that this formatting is ambiguous about the year. 
    --> Dates fromatted that way will be converted to the MySQL friendly date format `YYY-MM-DD`.
    
    
2. Dates are also formatted with the `MM-DD` format, fields encoded that way are:
     - `SEASON_BEGIN_DATE` formatted in `MM-DD`
     - `SEASON_END_DATE` formatted in `MM-DD`
     
     Note that those dates inform on year-recurring events, they occur every years. We keep them as raw text fields. Fine grained processing of those dates will be done by consumers applications.
     


## Load and format data

In [83]:
OUTSTANDING_PERFORM_BEGIN_DATE_idx = 44
PWS_DEACTIVATION_DATE_idx = 8
SOURCE_PROTECTION_BEGIN_DATE_idx = 42

In [126]:
df = pd.read_csv(join(PATH_TO_DATA_FOLDER, "DG_WATER_SYSTEM.csv"), sep=",", header=0, parse_dates=[PWS_DEACTIVATION_DATE_idx, SOURCE_PROTECTION_BEGIN_DATE_idx, OUTSTANDING_PERFORM_BEGIN_DATE_idx])



/Users/fpaupier/.envs/safe-water/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [128]:
df.head()

,WATER_SYSTEM.PWSID,WATER_SYSTEM.PWS_NAME,WATER_SYSTEM.NPM_CANDIDATE,WATER_SYSTEM.PRIMACY_AGENCY_CODE,WATER_SYSTEM.EPA_REGION,WATER_SYSTEM.SEASON_BEGIN_DATE,WATER_SYSTEM.SEASON_END_DATE,WATER_SYSTEM.PWS_ACTIVITY_CODE,WATER_SYSTEM.PWS_DEACTIVATION_DATE,WATER_SYSTEM.PWS_TYPE_CODE,...,WATER_SYSTEM.CITY_NAME,WATER_SYSTEM.ZIP_CODE,WATER_SYSTEM.COUNTRY_CODE,WATER_SYSTEM.STATE_CODE,WATER_SYSTEM.SOURCE_WATER_PROTECTION_CODE,WATER_SYSTEM.SOURCE_PROTECTION_BEGIN_DATE,WATER_SYSTEM.OUTSTANDING_PERFORMER,WATER_SYSTEM.OUTSTANDING_PERFORM_BEGIN_DATE,WATER_SYSTEM.CITIES_SERVED,WATER_SYSTEM.COUNTIES_SERVED
0,AR1900063,USCOE BSWP118 PONTIAC,N,AR,6,01-01,12-31,I,1983-06-01,TNCWS,...,LITTLE ROCK,72203,US,AR,NaN,NaT,NaN,NaT,Not Reported,Marion
1,AR1900071,USCOE BSWP126 HIGHWAY K,N,AR,6,01-01,12-31,I,1983-06-01,TNCWS,...,LITTLE ROCK,72203,US,AR,NaN,NaT,NaN,NaT,Not Reported,Marion
2,AR1900072,USCOE BSW127 LOWERY,N,AR,6,01-01,12-31,I,1983-06-01,TNCWS,...,LITTLE ROCK,72203,US,AR,NaN,NaT,NaN,NaT,Not Reported,Marion
3,AR1900075,USCOE GFW02 DAM SITE,N,AR,6,01-01,12-31,I,1983-06-01,TNCWS,...,LITTLE ROCK,72203,US,AR,NaN,NaT,NaN,NaT,Not Reported,Cleburne
4,AR1900076,USCOE GFW03 DAM SITE,N,AR,6,01-01,12-31,I,1983-06-01,TNCWS,...,LITTLE ROCK,72203,US,AR,NaN,NaT,NaN,NaT,Not Reported,Cleburne


### Sanitize booleans
The fields `IS_GRANT_ELIGIBLE_IND`, `IS_WHOLESALER_IND` and `IS_SCHOOL_OR_DAYCARE_IND` are text values `N` or `Y`. They should be casted as `Booleans` in the database to allow easily expressed queries.

In [129]:
df['WATER_SYSTEM.IS_GRANT_ELIGIBLE_IND'].dropna().unique()

array(['N', 'Y'], dtype=object)

In [130]:
df['WATER_SYSTEM.IS_WHOLESALER_IND'].dropna().unique()

array(['N', 'Y'], dtype=object)

In [131]:
df['WATER_SYSTEM.IS_SCHOOL_OR_DAYCARE_IND'].dropna().unique()

array(['N', 'Y'], dtype=object)

### Perform sanitization

Map `N` to `False` and `Y` to `True`.

In [132]:
df['WATER_SYSTEM.IS_GRANT_ELIGIBLE_IND'] = df['WATER_SYSTEM.IS_GRANT_ELIGIBLE_IND'].map({'N': False, 'Y': True})
df['WATER_SYSTEM.IS_WHOLESALER_IND'] = df['WATER_SYSTEM.IS_WHOLESALER_IND'].map({'N': False, 'Y': True})
df['WATER_SYSTEM.IS_SCHOOL_OR_DAYCARE_IND'] = df['WATER_SYSTEM.IS_SCHOOL_OR_DAYCARE_IND'].map({'N': False, 'Y': True});

Check sanitization output:

In [133]:
df['WATER_SYSTEM.IS_GRANT_ELIGIBLE_IND'].dropna().unique()

array([False,  True])

In [134]:
df['WATER_SYSTEM.IS_WHOLESALER_IND'].dropna().unique()

array([False,  True])

In [135]:
df['WATER_SYSTEM.IS_SCHOOL_OR_DAYCARE_IND'].dropna().unique()

array([False,  True])

## Save sanitized data

Save the sanitized dataset in a new csv file:

In [136]:
# Data will be saved in the `sanitized` folder.
sanitized_csv_file = join(PATH_TO_DATA_FOLDER, 'sanitized', 'WATER_SYSTEM.csv')

In [137]:
df.to_csv(sanitized_csv_file, sep=",", encoding='utf-8')

